In [1]:
%load_ext autoreload
%autoreload 2

import os
import napari

import sys
sys.path.append('..')

from tracks_interactions.widget.widget_main import TrackGardener

In [2]:
os.environ['NAPARI_PERFMON'] = '0'

from napari.settings import get_settings
settings = get_settings()
settings.experimental.async_
#settings.experimental.async_=True

False

In [4]:
# get access to the database
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

new_db_path = r'D:\kasia\tracking\E6_exp\double_segmentation_ultrack\Exp6_gardener_v6.db'
engine = create_engine(f'sqlite:///{new_db_path}')
session = sessionmaker(bind=engine)()

from sqlalchemy import text
session.execute(text("ANALYZE;"))
session.commit()

In [3]:
viewer = napari.Viewer()

track_gardener = TrackGardener(viewer)

viewer.window.add_dock_widget(track_gardener,area='right',name='Track Gardener')

In [6]:
from tracks_interactions.db.db_model import NO_PARENT, CellDB, TrackDB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyqtgraph as pg
from sqlalchemy.orm.attributes import flag_modified
import tracks_interactions.db.db_functions as fdb

In [30]:
cell_id = 39122
current_frame = 209

In [33]:
c=session.query(CellDB).filter(CellDB.track_id == cell_id).filter_by(t = current_frame).order_by(CellDB.t).all()
c

[210006237 from frame 209 with track_id 39122 at (4116,4520)]

In [29]:
c[0].signals

IndexError: list index out of range

In [37]:
cell_id = 20426
t = session.query(TrackDB).filter_by(track_id = cell_id).first()
t

Track 20426 from 43 to 80

In [36]:
session.query(CellDB.t).filter(CellDB.track_id == cell_id).all()

[(144,),
 (145,),
 (146,),
 (147,),
 (148,),
 (149,),
 (150,),
 (151,),
 (152,),
 (153,),
 (154,),
 (155,),
 (156,),
 (157,),
 (158,),
 (159,),
 (160,)]

In [97]:
viewer.camera.events.zoom.disconnect(track_gardener.navigation_widget.build_labels)
viewer.camera.events.center.disconnect(track_gardener.navigation_widget.build_labels)

In [84]:
for event_emitter in viewer.dims.events.emitters.values():
    event_emitter.disconnect()

In [72]:
ch = track_gardener.settings_window.channels_list[0]
channel_name = ch.get('name', 'Unnamed')
channel_path = ch.get('path', '')
channel_lut = ch.get('lut', 'green')

In [87]:
'Labels' in viewer.layers

True

In [85]:
import dask.array as da

channels_data_list = []
for ch in track_gardener.settings_window.channels_list:
    channel_name = ch.get('name', 'Unnamed')
    channel_path = ch.get('path', '')
    channel_lut = ch.get('lut', 'green')

    ch_list = []
    for level in range(1, 5):
        ch_list.append(da.from_zarr(channel_path, level))

    channels_data_list.append(ch_list)

    viewer.add_image(
        ch_list,
        name=channel_name,
        colormap=channel_lut,
        blending='additive',
        contrast_limits=[0, 2048],
    )
            

KeyError: "'Labels' is not in list"

In [73]:
viewer.add_image(track_gardener.settings_window.channels_data_list[0],name=channel_name,colormap=channel_lut,blending='additive',contrast_limits=[0, 2048])

<Image layer 'ch0 [1]' at 0x1d6aba1e800>

In [70]:
track_gardener.settings_window.channels_list

[{'name': 'ch0',
  'path': 'D:\\kasia\\tracking\\E6_exp\\E6_C0.zarr',
  'lut': 'green'},
 {'name': 'ch1',
  'path': 'D:\\kasia\\tracking\\E6_exp\\E6_C1.zarr',
  'lut': 'red'}]

In [64]:
for event_emitter in viewer.dims.events.emitters.values():
    event_emitter.disconnect()

layers_list = [x.name for x in viewer.layers]

for layer in layers_list:
    viewer.layers.remove(layer)

In [57]:
track_gardener.settings_window.channels_data_list

[[dask.array<from-zarr, shape=(241, 8396, 8401), dtype=uint16, chunksize=(10, 512, 512), chunktype=numpy.ndarray>,
  dask.array<from-zarr, shape=(241, 4198, 4200), dtype=uint16, chunksize=(10, 512, 512), chunktype=numpy.ndarray>,
  dask.array<from-zarr, shape=(241, 2099, 2100), dtype=uint16, chunksize=(10, 512, 512), chunktype=numpy.ndarray>,
  dask.array<from-zarr, shape=(241, 1049, 1050), dtype=uint16, chunksize=(10, 512, 512), chunktype=numpy.ndarray>]]

In [26]:
import yaml
from tracks_interactions.widget.signal_graph_widget import CellGraphWidget
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# get access to the database
new_db_path = r'D:\kasia\tracking\E6_exp\double_segmentation_ultrack\Exp6_gardener_v6.db'
engine = create_engine(f'sqlite:///{new_db_path}')
session = sessionmaker(bind=engine)()

filePath = r'D:\kasia\tracking\E6_exp\sample_data\config.yaml'

with open(filePath, 'r') as file:
    config = yaml.safe_load(file)
    graph_list = config.get('graphs', [])
    cell_tags = config.get('cell_tags', [])

signal_list = ['area','ch0_nuc','ch0_cyto','ch1_nuc','ch1_cyto']

ind = 2
for gr in graph_list:
    graph_name = gr.get('name', 'Unnamed')
    graph_signals = gr.get('signals', [])
    graph_colors = gr.get('colors', [])

    graph_widget = CellGraphWidget(viewer,session,signal_list,signal_sel_list=graph_signals,color_sel_list=graph_colors,tag_dictionary=cell_tags)
    viewer.window.add_dock_widget(graph_widget,area='right')
    #track_gardener.tab2.layout().addWidget(graph_widget, ind, 0)
    #ind += 1